#### 1. 불러오기

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train = pd.read_csv('./train.csv')
train_x = train.drop(['ID', 'vehicle_speed(km/h)'], axis=1)
train_y = train['vehicle_speed(km/h)']

test_x = pd.read_csv('./test.csv')
test_x = test_x.drop(['ID'], axis = 1)

In [ ]:
train.head()

#### 2. EDA

데이터분석을 위해 가장 먼저 해야할 일은 바로 데이터를 살펴보는 것입니다. <br>
주어진 데이터를 살펴보며 데이터의 모양과 대략적인 내용을 파악하고, 어떤 분석이 가능한지 확인하여 방향을 잡는 과정이 필요합니다.

#### 2-1. 데이터 자료형과 통계량 확인

info(), describe() 메소드를 활용하여 데이터의 자료형(Dtype)과 이상치나 결측치의 존재 유무를 확인합니다.

In [ ]:
train_x.info()

In [ ]:
# 각 변수들의 데이터 통계량을 확인합니다.
train_x.describe()

#### 2-2. 데이터 시각화

변수들 간 상관관계를 보여주는 pairplot, 연속형 변수의 분포를 보여주는 distplot을 이용해서 EDA를 진행합니다.

In [ ]:
plt.figure(figsize=(12,12))
sns.pairplot(train)
plt.show()

#'brake_cmd', 'gear_position'이 범주형 변수임을 확인

In [ ]:
train_x.columns

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(20,16))

sns.distplot(x=train['X1'], ax=axes[0][0]).set_title('X1')
sns.distplot(x=train['X2'], ax=axes[0][1]).set_title('X2')
sns.distplot(x=train['acceleration_cmd'], ax=axes[0][2]).set_title('acceleration_cmd')
sns.distplot(x=train['brake_cmd'], ax=axes[0][3]).set_title('brake_cmd')
sns.distplot(x=train['gear_position'], ax=axes[1][0]).set_title('gear_position')
sns.distplot(x=train['steering_angle(deg)'], ax=axes[1][1]).set_title('steering_angle(deg)')
sns.distplot(x=train['lateral_acceleration(m/s^2)'], ax=axes[1][2]).set_title('lateral_acceleration(m/s^2)')
sns.distplot(x=train['longitudinal_acceleration(m/s^2)'], ax=axes[1][3]).set_title('longitudinal_acceleration(m/s^2)')
sns.distplot(x=train['yaw_rate(deg/s)'], ax=axes[2][0]).set_title('yaw_rate(deg/s)')
sns.distplot(x=train['linear_acceleration_x(m/s^2)'], ax=axes[2][1]).set_title('linear_acceleration_x(m/s^2)')
sns.distplot(x=train['UTM_x'], ax=axes[2][2]).set_title('UTM_x')
sns.distplot(x=train['UTM_y'], ax=axes[2][3]).set_title('UTM_y')
sns.distplot(x=train['vehicle_speed(km/h)'], ax=axes[3][0]).set_title('vehicle_speed(km/h)')

plt.show()

#['X1', 'X2', 'MC', 'lateral_acceleration(m/s^2)', 'linear_acceleration_x(m/s^2)'] 등은 분포가 중앙으로 몰려있는 형태
#['acceleration_cmd', 'vehicle_speed(km/h)'] 등은 분포가 한 쪽으로 치우친 형태

#### 2-3. 다중공선성과 상관관계 확인

하나의 독립변수가 다른 여러 개의 독립변수들로 잘 예측되는 다중공선성을 확인하기 위해, 분산팽창계수(VIF)를 구하고, 히트맵을 사용하여 각 변수 간의 상관계수를 시각적으로 확인하겠습니다. <br>
다중공선성이 있는 경우, 상관관계가 높은 변수 중 하나를 제거하거나, 주성분 분석(PCA)과 같은 차원 축소 기법을 사용하여 변수를 줄입니다.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
feature = train_x

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(feature.values, i) for i in range(feature.shape[1])]
vif['features'] = feature.columns
picked = list(vif[vif['VIF Factor'] > 10].features)
print(picked)

vif

In [ ]:
temp = train_x.corr()
#그림 사이즈 지정
fig, ax = plt.subplots(figsize=(12,12))

#삼각형 마스크를 만든다(위 쪽 삼각형에 True, 아래쪽 삼각형에 False)
mask = np.zeros_like(temp, dtype=bool)
mask[np.triu_indices_from(mask)] = True

#히트맵 그리기
sns.heatmap(temp,
            cmap='RdBu_r',
            annot=True,              #실제값을 표시한다.
            mask=mask,               #표시하지 않을 마스크 부분을 지정한다.
            linewidths = .5,         #경계면 실선으로 구분하기
            cbar_kws={'shrink': .5}, #컬러비 크기 절반으로 줄이기
            vmin = -1, vmax = 1)     #컬러비 범위 -1 ~ 1

plt.show()

#### 2-4. 인사이트: 변수 선택

엄밀한 기준은 없으나 보통 분산팽창계수(VIF)가 10보다 크면 다중공선성이 있다고 판단합니다. (5를 기준으로 삼기도 함.)<br>
'yaw_rate(deg/s)', 'lateral_acceleration(m/s^2)' 변수는 분산 팽창 계수가 5이상입니다. <br>
이번 분석에서는 'yaw_rate(deg/s)' 변수를 사용하지 않겠습니다. <br>
또한, 전부 동일한 값을 갖는 'brake_cmd' 변수 또한 사용하지 않겠습니다. <br>
drop() 메소드의 columns 인자에 제거할 변수를 리스트에 담아 넣어줍니다.

In [ ]:
train_x = train_x.drop(columns=['yaw_rate(deg/s)', 'brake_cmd'], axis=1)
test_x = test_x.drop(columns=['yaw_rate(deg/s)', 'brake_cmd'], axis=1)

#### 3. 데이터 전처리

컴퓨터는 숫자만 계산 가능하기 때문에 문자형으로 이루어진 범주형 변수들을 분석에 사용하려면 숫자형으로 변환해 주어야 합니다. <br>
본 데이터셋에는 문자형 변수가 이미 숫자형으로 표현되어 있기 때문에 이 과정을 생략하도록 합니다.

In [ ]:
train_x['gear_position'].value_counts()

#### 3-1. 스케일링

데이터 모델링에서 스케일링은 다차원의 값들을 비교 분석하기 쉽게 만들어 자료의 오버플로우(overflow)나 언더플로우(underflow)를 방지하고 최적화 과정에서의 안정성 및 수렴 속도를 향상시킬 수 있습니다.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#각 변수들의 분포를 고려하여 스케일링을 진행합니다.
#분포가 중앙으로 몰려있는 경우, 최솟값을 0, 최댓값을 1로 변환하는 MinMaxScaler 적용
#분포가 한 쪽으로 치우친 형태의 경우, 평균과 표준편차를 이용한 StandardScaler 적용
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

minmax_columns = ['X1', 'X2', 'steering_angle(deg)', 'longitudinal_acceleration(m/s^2)', 'lateral_acceleration(m/s^2)', 'UTM_x', 'UTM_y']
standard_columns = ['acceleration_cmd']

scaled_train_x = train_x.copy()
scaled_train_x[minmax_columns] = minmax_scaler.fit_transform(train_x[minmax_columns])
scaled_train_x[standard_columns] = standard_scaler.fit_transform(train_x[standard_columns])

scaled_test_x = test_x.copy()
# [주의] 단, 테스트데이터에는 transform만 적용
scaled_test_x[minmax_columns] = minmax_scaler.transform(test_x[minmax_columns])
scaled_test_x[standard_columns] = standard_scaler.transform(test_x[standard_columns])

#### 4. 분석 모델 설계

선형 모델인 Linear Regression, 트리 기반 모델 중 배깅 기법을 적용한 Random Forest와 ExtraTrees, 트리 기반 모델 중 부스팅 기법을 적용한 XGBoost와 Light GBM을 사용하여 어떤 모델이 좋은 성능을 보여주는지 실험합니다.

In [ ]:
lr = LinearRegression()
rf = RandomForestRegressor()
et = ExtraTreesRegressor()
xgb = XGBRegressor()
lgbm = LGBMRegressor()

#### 4-1. 분석 모델 성능 평가

성능을 평가하기 위해 학습 데이터셋에서 20%를 활용해 검증 데이터 셋을 만듭니다. <br>
이때, 분할된 학습 데이터를 사용하여 최종 모델을 학습시키는 것은 검증 데이터셋으로 할당한 20% 만큼의 데이터 손실이 발생한다는 점을 유의하시기 바랍니다.

In [ ]:
train_x_splited, val_x, train_y_splited, val_y = train_test_split(scaled_train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
lr.fit(train_x_splited, train_y_splited)
lr_pred = lr.predict(val_x)
print('Linear Regression 모델의 MAE : ', np.sqrt(mean_squared_error(val_y, lr_pred)))

rf.fit(train_x_splited, train_y_splited)
rf_pred = rf.predict(val_x)
print('Random Forest 모델의 MAE : ', np.sqrt(mean_squared_error(val_y, rf_pred)))

et.fit(train_x_splited, train_y_splited)
et_pred = et.predict(val_x)
print('Extra Trees 모델의 MAE : ', np.sqrt(mean_squared_error(val_y, et_pred)))

xgb.fit(train_x_splited, train_y_splited)
xgb_pred = xgb.predict(val_x)
print('XGBoost 모델의 MAE : ', np.sqrt(mean_squared_error(val_y, xgb_pred)))

lgbm.fit(train_x_splited, train_y_splited)
lgbm_pred = lgbm.predict(val_x)
print('LGBM 모델의 MAE : ', np.sqrt(mean_squared_error(val_y, lgbm_pred)))

#### 4-2. GridSearchCV(Gridsearch Cross Validation)를 활용한 하이퍼파라미터 튜닝

이제 검증 데이터셋을 이용한 평가에서 가장 좋은 성능을 보인 Extra Trees, LGBM 모델의 하이퍼파라미터 튜닝을 진행하기 위해 Gridsearch CV를 활용합니다.<br>
많은 조합을 한 번에 확인할 수도 있지만, 시간관계상 한 개의 조합만을 확인해보겠습니다.

In [ ]:
param_grid = [
    {'n_estimators' : [100],
     'min_samples_leaf' : [1],
     'random_state' : [0]}
]

grid_search = GridSearchCV(et,
                           param_grid,
                           cv=3,
                           scoring='neg_root_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)

grid_search.fit(scaled_train_x, train_y)

print('최적의 하이퍼파라미터 : ', grid_search.best_params_)

#### 5. 모델 학습

In [ ]:
et = ExtraTreesRegressor(**grid_search.best_params_) #최적 하이퍼파라미터를 불러와 모델을 구성
et.fit(scaled_train_x, train_y)

#### 6. 예측값 생성

In [ ]:
pred = et.predict(scaled_test_x)

#### 7. 제출 파일 생성

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission.head(5)

In [ ]:
submission['vehicle_speed(km/h)'] = pred
submission.head(5)

In [ ]:
submission.to_csv('./et_submit.csv', index = False)